In [1]:
import pandas as pd

In [2]:
df=pd.read_excel('/Users/gavin/Documents/metis/coursework/project_3/data_sources/NFL_week_data.xlsx')

In [3]:
df.head()

,schedule_date,schedule_season,schedule_week,target,target_score,opponent_score,opponent,target_spread,opponent_spread,target_games_played,opponent_games_played,target_off_points_scored,opponent_off_points_scored,target_def_points_against,opponent_def_points_against
0,2015-09-10,2015,1,New England Patriots,28,21,Pittsburgh Steelers,-7.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-09-10,2015,1,Pittsburgh Steelers,21,28,New England Patriots,7.0,-7.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-09-13,2015,1,Arizona Cardinals,31,19,New Orleans Saints,-2.5,2.5,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-09-13,2015,1,Buffalo Bills,27,14,Indianapolis Colts,1.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-09-13,2015,1,Chicago Bears,23,31,Green Bay Packers,6.5,-6.5,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df['target_off_points_game'] = df['target_off_points_scored']/df['target_games_played']
df['opponent_off_points_game'] = df['opponent_off_points_scored']/df['opponent_games_played']
df['target_def_points_against_game'] = df['target_def_points_against']/df['target_games_played']
df['opponent_def_points_against_game'] = df['opponent_def_points_against']/df['opponent_games_played']

In [5]:
df['target_score_spread_adj'] = df['target_score']+df['target_spread']
df['beat_spread']=0
df.loc[df['target_score_spread_adj']>df['opponent_score'], 'beat_spread'] = 1

In [6]:
df['off_spread_impactor'] = 150-df['target_spread']
df['def_spread_impactor'] = 150-df['target_spread']

In [7]:
df['off_points_SI'] = df['target_off_points_game']/df['opponent_off_points_game']/df['off_spread_impactor']*100
df['def_points_SI'] = df['target_def_points_against_game']/df['opponent_def_points_against_game']/df['def_spread_impactor']*100

In [8]:
df['target_favorite'] = 0
df.loc[df['target_spread']<0,'target_favorite']=1

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

%matplotlib inline

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

In [10]:
df_run = df[df['schedule_week']>=3].copy()

In [11]:
df_run.head()

,schedule_date,schedule_season,schedule_week,target,target_score,opponent_score,opponent,target_spread,opponent_spread,target_games_played,...,opponent_off_points_game,target_def_points_against_game,opponent_def_points_against_game,target_score_spread_adj,beat_spread,off_spread_impactor,def_spread_impactor,off_points_SI,def_points_SI,target_favorite
64,2015-09-24,2015,3,New York Giants,32,21,Washington Redskins,-3.0,3.0,2.0,...,17.0,25.5,13.5,29.0,1,153.0,153.0,0.884275,1.234568,1
65,2015-09-24,2015,3,Washington Redskins,21,32,New York Giants,3.0,-3.0,2.0,...,23.0,13.5,25.5,24.0,0,147.0,147.0,0.502810,0.360144,0
66,2015-09-27,2015,3,Arizona Cardinals,47,7,San Francisco 49ers,-7.0,7.0,2.0,...,19.0,21.0,23.0,40.0,1,157.0,157.0,1.324170,0.581556,1
67,2015-09-27,2015,3,Baltimore Ravens,24,28,Cincinnati Bengals,-1.5,1.5,2.0,...,28.5,28.0,16.0,22.5,0,151.5,151.5,0.532685,1.155116,1
68,2015-09-27,2015,3,Carolina Panthers,27,22,New Orleans Saints,-9.5,9.5,2.0,...,19.0,13.0,28.5,17.5,0,159.5,159.5,0.725953,0.285981,1


In [12]:
df_run = df_run[(df_run['def_points_SI']!=np.inf) & (df_run['off_points_SI']!=np.inf)]
df_run.reset_index(drop=True, inplace=True)

In [13]:
X = df_run[['off_points_SI', 'def_points_SI', 'target_favorite']]
y = df_run['beat_spread']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state=42)

In [ ]:
ss = StandardScaler()
xscale_train = ss.fit_transform(X_train)
xscale_test = ss.fit_transform(X_test)

In [ ]:
lr = LogisticRegression()
lr.fit(xscale_train, y_train)
lr.score(xscale_test, y_test)

In [ ]:
scores =[]
for i in range(3, 17):
    df_run = df[df['schedule_week']>=2].copy()

    df_run = df_run[(df_run['def_points_SI']!=np.inf) & (df_run['off_points_SI']!=np.inf)]
    df_run.reset_index(drop=True, inplace=True)

    df_run = df_run[(df_run['def_points_SI']!=np.inf) & (df_run['off_points_SI']!=np.inf)]
    df_run.reset_index(drop=True, inplace=True)

    X = df_run[['targ_off_points_SI', 'targ_def_points_SI']]
    y = df_run['beat_spread']

    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.5, random_state=42)

    ss = StandardScaler()
    xscale_train = ss.fit_transform(X_train)
    xscale_test = ss.fit_transform(X_test)

    lr = LogisticRegression()
    lr.fit(xscale_train, y_train)
    score = lr.score(xscale_test, y_test)
    scores.append([i, score])
    

In [ ]:
scores

In [ ]:
randomforest = RandomForestClassifier()
randomforest.fit(X_train, y_train)
randomforest.score(X_test, y_test)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
nearest = knn.fit(X_train, y_train)
print(nearest.score(xscale_train, y_train))
print(nearest.score(xscale_test, y_test))

In [14]:
#######################
# standard code block #
#######################

# see https://ipython.readthedocs.io/en/stable/interactive/magics.html
%pylab inline

# sets backend to render higher res images
%config InlineBackend.figure_formats = ['retina']

#######################
#       imports       #
#######################
import pandas as pd
import seaborn as sns
# from sklearn.datasets import california_housing
# from sklearn.datasets import load_breast_cancer
# from sklearn.metrics import accuracy_score
import xgboost as xgb

sns.set_style("whitegrid")

Populating the interactive namespace from numpy and matplotlib


In [15]:
#Split data into 3: 60% train, 20% validation, 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=2)

In [23]:
gbm = xgb.XGBClassifier( 
                        n_estimators=30000,
                        max_depth=4,
                        objective='binary:logistic', #new objective
                        learning_rate=.05, 
                        subsample=.8,
                        min_child_weight=3,
                        colsample_bytree=.8
                       )

eval_set=[(X_train,y_train),(X_val,y_val)]
fit_model = gbm.fit( 
                    X_train, y_train, 
                    eval_set=eval_set,
                    eval_metric='auc', #new evaluation metric: classification error (could also use AUC, e.g.)
                    early_stopping_rounds=50,
                    verbose=True
                   )

accuracy_score(y_test, gbm.predict(X_test, ntree_limit=gbm.best_ntree_limit)) 

[0]	validation_0-auc:0.57729	validation_1-auc:0.50681
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 50 rounds.
[1]	validation_0-auc:0.59949	validation_1-auc:0.48445
[2]	validation_0-auc:0.61869	validation_1-auc:0.46646
[3]	validation_0-auc:0.61918	validation_1-auc:0.47281
[4]	validation_0-auc:0.61999	validation_1-auc:0.48475
[5]	validation_0-auc:0.61587	validation_1-auc:0.48435
[6]	validation_0-auc:0.63595	validation_1-auc:0.47295
[7]	validation_0-auc:0.63887	validation_1-auc:0.47920
[8]	validation_0-auc:0.63731	validation_1-auc:0.48460
[9]	validation_0-auc:0.64180	validation_1-auc:0.48399
[10]	validation_0-auc:0.64608	validation_1-auc:0.48253
[11]	validation_0-auc:0.64608	validation_1-auc:0.48264
[12]	validation_0-auc:0.65021	validation_1-auc:0.49143
[13]	validation_0-auc:0.65040	validation_1-auc:0.49167
[14]	validation_0-auc:0.66151	validation_1-auc:0.48480
[15]	validation_0-auc:0.6702

0.5439330543933054

In [22]:
import pickle
outfile = open('nflxgb.pkl', 'wb')
pickle.dump(gbm, outfile)
outfile.close()

In [20]:
infile = open('nflxgb.pkl', 'rb')
model = pickle.load(infile)

In [21]:
accuracy_score(y_test, model.predict(X_test, ntree_limit=gbm.best_ntree_limit))

0.5418410041841004

In [ ]:
df_trees = X_test.reset_index(drop=True)

In [ ]:
df_trees['beat_spread'] = y_test.reset_index(drop=True)

In [ ]:
df_trees['guess'] = gbm.predict(X_test, ntree_limit=gbm.best_ntree_limit)

In [ ]:
df_trees[df_trees['guess']==1].beat_spread.sum()

In [ ]:
df_trees[df_trees['guess']==1]

In [ ]:
98/195

In [ ]:
df_trees.head()

In [ ]:
df_

In [ ]:
df_trees[df_trees['guess']==1].shape

In [ ]:
df_trees[df_trees['guess']==1].beat_spread.sum()

In [ ]:
410/737

In [ ]:
df['spread_impactor'] = (100-df[]

In [ ]:
from sklearn.naive_bayes import BernoulliNB, MultinomialNB, GaussianNB

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

In [ ]:
nb_bern = BernoulliNB()
nb_bern.fit(xscale_train, y_train)
nb_bern.score(xscale_test, y_test)

In [ ]:
nb = GaussianNB()
nb.fit(X_train, y_train)
nb.score(X_test, y_test)

In [ ]:
nb.fit(X)

In [ ]:
# Let's look at what lm1 did
import matplotlib.pyplot as plt

feature = X_test
prediction_hard = nb_mult.predict(feature)

sf_mask = (y_test == 1).values

plt.plot(feature[sf_mask], prediction_hard[sf_mask], 'ro', label='True', alpha=0.2)
plt.plot(feature[~sf_mask], prediction_hard[~sf_mask], 'bo', label='False', alpha=0.2)
plt.ylim(-0.05, 1.05)
plt.xlabel('score')
plt.ylabel('Prediction (1=Win)')
plt.title('Logistic regression prediction for Wins V Loss')
plt.legend(loc='lower right');

In [ ]:
df_run['prob_true']=nb_mult.predict_proba(X)[:,1]

In [ ]:
df_run[(df_run['prob_true']>.5)]

In [ ]:
df_run[(df_run['prob_true']>.6)].beat_spread.sum()

In [ ]:
70/129

In [ ]:
df_run.rename(columns = {'off_points_SI':'target_off_points_SI', 'def_points_SI':'target_def_points_SI', 'prob_true':'target_prob_true'}, inplace=True)

In [ ]:
df_run.head(30)

In [ ]:
df_run['schedule_season_str']=df_run['schedule_season'].apply(str)
df_run['schedule_week_str']=df_run['schedule_week'].apply(str)
# df_run['identifier'] = df_run['schedule_season_str']+_+(df_run['schedule_week_str'])+_+df_run['target']+_+df_run['opponent']

In [ ]:
df_run.drop(['schedule_season_str', 'schedule_week_str', 'identifier'], axis=1, inplace=True)

In [ ]:
df_run['identifier'] = df_run['schedule_date'].apply(str)+'_'+df_run['target']+'_'+df_run['opponent']

In [ ]:
df_run['identifier1']=df_run['identifier'].str.split('_')

In [ ]:
df_run['identifier1']=df_run['identifier']

In [ ]:
df_run['identifier'] = df_run.identifier1.apply(sorted)

In [ ]:
df_run['opponent_prob']=0

In [ ]:
df_run['opponent_prob'] = df_run.loc[(df_run['identifier']==df_run['identifier1'])& (df_run['target']==df_run['opponent']),'target_prob_true']

In [ ]:
df_run.dtypes

In [ ]:
df_run.head()